In [7]:
from importlib import reload
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='whitegrid')
import ard_logging
import logging
ard_logging.init(force_debug=False)
LOG = logging.getLogger(__name__)
LOG.info('Hello from notebook')

import ard_fit_pipelines as fp
reload(fp.core); reload(fp.transforms); reload(fp)

logging.getLogger('ard_fit_pipelines').setLevel(logging.DEBUG)

2022-06-18 20:09:41,024:    INFO:      ard_logging: MainProcess 143: Logging initialized.
2022-06-18 20:09:41,026:    INFO:         __main__: MainProcess 143: Hello from notebook


In [2]:
fp.transforms.KeepColumns?

Init signature: fp.transforms.KeepColumns(cols: 'str | hp | Iterable[str | hp]') -> None
Docstring:     
Abstract base clase of all Transforms that operate on a parameterized list
of columns.
Init docstring: Method generated by attrs for class ColumnsTransform.
File:           ~/projects/ardua/codebases/local/ard-fit-pipelines/src/ard_fit_pipelines/transforms.py
Type:           ABCMeta
Subclasses:     


In [3]:
df = sns.load_dataset('diamonds')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
reload(fp.transforms)
t = fp.transforms.CopyColumns(cols=['price', 'x'], dest_cols=['price_train', 'x_copy'])
display(t)
display(t.apply(df.head()))

CopyColumns(cols=('price', 'x'), dest_cols=('price_train', 'x_copy'))

2022-06-18 20:07:51,808:   DEBUG: ard_fit_pipelines.core: MainProcess 143: Applying Transform to 5 rows of data: CopyColumns(cols=('price', 'x'), dest_cols=('price_train', 'x_copy'))


,carat,cut,color,clarity,depth,table,price,x,y,z,price_train,x_copy
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,326,3.95
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,326,3.89
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,327,4.05
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,334,4.20
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,335,4.34


In [9]:
t = fp.transforms.CopyColumns(cols=fp.core.hp('train_response_col'), dest_cols=['price_train'])
t.bind_params({
    'train_response_col': 'price'
})
t

CopyColumns(cols=(hp(name='train_response_col'),), dest_cols=('price_train',))

In [10]:
t.bound_params

{'cols': (hp(name='train_response_col'),), 'dest_cols': ('price_train',)}

In [30]:
t.bound_params = 1
#object.__setattr__(t, 'bound_params', 1)

I recently started using the `python-attrs` library for the first time, and I think it's really neat. As a new user of it, the only thing that has stood out to me as questionable about its design is that `attrs.define` defaults to `slots=True`, so that slotted classes are created by default. I'm curious if anyone else agrees with me that this is a bad default, or what the counterargument is.

It think it's bad because it totally changes the semantics of OOP unexpectedly... inheritance is often broken by slots (even without using multiple inheritance), attributes cannot be added to an instance after instantiation (unlike ordinary classes), and attributes can become unexpected read-only. As a new user who was sold on trying attrs by its Philosophy (in the documentation) of "it's about regular classes", "be light on API impact", and "no surprises," these unexpected changes to the core semantics of classes are confusing and annoying. They even say in their own documentation:

> slotted classes

> A class whose instances have no `object.__dict__` attribute and define their attributes in a `object.__slots__` attribute instead. In attrs, they are created by passing slots=True to @attr.s (and are on by default in attr.define/attr.mutable/attr.frozen).

> Their main advantage is that they use less memory on CPython 1 and are slightly faster.

> However they also come with several possibly surprising gotchas

... so why is it the default in the "new API" ?